In [1]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [10]:
import os 
os.chdir("/content/drive/My Drive/Gutenberg_books")


In [17]:
import re
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from sklearn.preprocessing import LabelEncoder

In [4]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [12]:
# Read the csv file into pandas dataframe
data = pd.read_csv("master996.csv",encoding = "ISO-8859-1")

#Replace book id with the actual name present in html so as to iterate through html files
data['book_id'] = data['book_id'].str.replace('.epub', '-content.html', case = False)
book_id_array = data['book_id'].to_numpy()
#print(book_id_array[0])

genre = ['guten_genre']

In [13]:
#Convert each genre of data dataframe into numbers
for x in genre:
    le = LabelEncoder()
    le.fit(list(data[x].values))
    data[x] = le.transform(list(data[x]))
print('******************Dataframe after converting Genre as numbers**********************')
print(data)

wordArray = []


******************Dataframe after converting Genre as numbers**********************
                                             Book_Name  ...               Author_Name
0    The Mystery of the Boule Cabinet: A Detective ...  ...  Stevenson| Burton Egbert
1                                            The Pupil  ...              James| Henry
2                                       At Love's Cost  ...          Garvice| Charles
3                               The Heart of the Range  ...  White| William Patterson
4                          The Worshipper of the Image  ...     Gallienne| Richard Le
..                                                 ...  ...                       ...
991                                  David Copperfield  ...          Dickens| Charles
992                                         Hard Times  ...          Dickens| Charles
993                          Memoirs of Shelock Holmes  ...             Connan| Doyle
994                    The Mysterious Affair at Styles  

In [14]:
#Iterate the documenta nd take all the words to fit into model

index = 0

#Iterate through each .html files and read the contents and append to our exisitng dataframe
while index < len(book_id_array):
	#Open the HTML file abd read the contents for tokenizing
	#Replace the path with the folder where html files are present
	#HTMLfile = open('E:\\OVGU\\Sem2_Summer2020\\Subjects\\ATML\\Semester_Project\\starter\\Gutenberg_English_Fiction_1k\\Gutenberg_19th_century_English_Fiction\\'+book_id_array[index],'r',encoding='utf-8')
	HTMLfile = open('/content/drive/My Drive/Gutenberg_books/Gutenberg_19th_century_English_Fiction/'+book_id_array[index],'r',encoding='utf-8')

	content = HTMLfile.read()
	r = re.compile('<.*?>')
	content_removed_tags = re.sub(r, '', content)

	#Pick pre-defineds Stop words from nltk lib
	stop_words = list(stopwords.words('english'))

	# Tokenize the text
	tokens = word_tokenize(content_removed_tags)
	# Convert into lower case
	tokens = [w.lower() for w in tokens]
	print('Document ',index,':')
	print('After Tokenizing: ',len(tokens))

	#Remove non alphabetic words/Removing punctuations
	words = [word for word in tokens if word.isalpha()]
	print('After Removing Punctuations: ',len(words))
	#Removing Stopwords
	words = [w for w in words if not w in stop_words]
	print('After Removing Stopwords: ',len(words))
	
	wordArray.insert(index,words)
	index = index+1

Document  0 :
After Tokenizing:  86326
After Removing Punctuations:  67441
After Removing Stopwords:  30231
Document  1 :
After Tokenizing:  20930
After Removing Punctuations:  17350
After Removing Stopwords:  7837
Document  2 :
After Tokenizing:  183689
After Removing Punctuations:  148363
After Removing Stopwords:  66654
Document  3 :
After Tokenizing:  115440
After Removing Punctuations:  87350
After Removing Stopwords:  44130
Document  4 :
After Tokenizing:  21263
After Removing Punctuations:  17574
After Removing Stopwords:  8311
Document  5 :
After Tokenizing:  32991
After Removing Punctuations:  27673
After Removing Stopwords:  12957
Document  6 :
After Tokenizing:  50583
After Removing Punctuations:  39742
After Removing Stopwords:  19149
Document  7 :
After Tokenizing:  94246
After Removing Punctuations:  71419
After Removing Stopwords:  36996
Document  8 :
After Tokenizing:  85087
After Removing Punctuations:  63954
After Removing Stopwords:  32855
Document  9 :
After Tokeniz

In [19]:
#data['tokens'] contains tokens present in each document
data['tokens'] = wordArray
#Convert the list into strings, column should have tokens seperated by space
data['list_string'] = data['tokens'].apply(lambda x: ' '.join(map(str, x)))

print('******************Dataframe after appending Tokens**********************')
#Dataframe after appending the tokens of respective books into data['list_string']
print(data)


******************Dataframe after appending Tokens**********************
                                             Book_Name  ...                                        list_string
0    The Mystery of the Boule Cabinet: A Detective ...  ...  detective story hello said took receiver desk ...
1                                            The Pupil  ...  edition first published text follows definitiv...
2                                       At Love's Cost  ...  moment never fully realised great ass man thin...
3                               The Heart of the Range  ...  rider golden bar hidden trails lynch lawyers o...
4                          The Worshipper of the Image  ...  worshipper image evening wood still dreaming b...
..                                                 ...  ...                                                ...
991                                  David Copperfield  ...  find easy get sufficiently far away book first...
992                                    

In [20]:
#Generating Feature vectors using Bag of words model
vectorizer = CountVectorizer()
vectors = vectorizer.fit_transform(data['list_string'])
print('************************Vectors Shape*******************************')
print(vectors.shape)

X_train, X_test, y_train, y_test = train_test_split(vectors, data['guten_genre'], test_size=0.3)
print('*******************Train and Test Set Size**************************')
print (X_train.shape)
print (y_train.shape)
print (X_test.shape)
print (y_test.shape)

************************Vectors Shape*******************************
(996, 157616)
*******************Train and Test Set Size**************************
(697, 157616)
(697,)
(299, 157616)
(299,)


In [24]:
clf = MultinomialNB(alpha=.45)
clf.fit(X_train, y_train)
pred = clf.predict(X_test)

print('****************Evaluation Metrics (average = macro) for test data original***************')
print('F1 Score: ',metrics.f1_score(y_test, pred, average='macro'))
print('Precision Score: ', metrics.precision_score(y_test, pred, average='macro'))
print('Recall Score: ', metrics.recall_score(y_test, pred, average='macro'))
print('Accuracy: ', metrics.accuracy_score(y_test, pred))

****************Evaluation Metrics (average = macro) for test data original***************
F1 Score:  0.2525099340385964
Precision Score:  0.24489962651727362
Recall Score:  0.26815776815776815
Accuracy:  0.8093645484949833


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [25]:
print('*******************Evaluation Metrics (average = weighted) for test data original*****************************')
print('F1 Score: ',metrics.f1_score(y_test, pred, average='weighted'))
print('Precision Score: ', metrics.precision_score(y_test, pred, average='weighted'))
print('Recall Score: ', metrics.recall_score(y_test, pred, average='weighted'))
print('Accuracy: ', metrics.accuracy_score(y_test, pred))

*******************Evaluation Metrics (average = weighted) for test data original*****************************
F1 Score:  0.7884527546670997
Precision Score:  0.770694448347901
Recall Score:  0.8093645484949833
Accuracy:  0.8093645484949833


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
